In [ ]:
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, anderson, jarque_bera, pearsonr, spearmanr
from scipy.stats import chi2_contingency
import math


def check_data(data_df):
    pd.set_option('display.max_columns', None)

    print('\033[1m' + 'Изучим исходные данные' + '\033[0m')
    print(data_df.info())
    print(data_df.shape)

    missed_cells = data_df.isnull().sum().sum() / (data_df.shape[0] *
                                                   (data_df.shape[1] - 1))
    missed_rows = sum(data_df.isnull().sum(axis=1) > 0) / data_df.shape[0]
    print('\033[1m' + "\nПроверка пропусков" + '\033[0m')
    print("Количество пропусков: {:.0f}".format(data_df.isnull().sum().sum()))
    # print("Количество пропусков2: \n", data_df.isna().sum())
    print("Доля пропусков {:.1%}".format(missed_cells) + '\033[0m')
    print("Доля строк, содержащих пропуски {:.1%}".format(missed_rows))

    # duplicates
    print('\033[1m' + "\nПроверка на дупликаты" + '\033[0m')
    print('Количество полных дупликатов: ', data_df.duplicated().sum())
    duplicateRows = data_df[data_df.duplicated()]
    print(duplicateRows)

    # data
    print('\033[1m' + "\nПервые 5 строчек датасета" + '\033[0m')
    print(data_df.head())  # tail(7)

    print('\033[1m' + '\nОписание количественных данных:' + '\033[0m')
    print(data_df.describe().T)

    print('\033[1m' + '\nОписание категориальных данных:' + '\033[0m')
    print(data_df.describe(include='object').T)

    print(
        '\033[1m' + '\nВывод уникальных значений по каждому категориальному признаку:' + '\033[0m')
    df_object = data_df.select_dtypes(include='object').columns

    for i in df_object:
        print('\033[1m' + '_' + str(i) + '\033[0m')
        print(data_df[i].value_counts())


def plot_hist(data, col_column, filename='project.png'):
    '''
    Функция отрисовки гистограмм и ящика с усами для количесвтенных переменных.
    На вход: исходная таблица и список количественных переменных.
    На выходе: графики
    '''
    rows = len(col_column)
    f, ax = plt.subplots(rows, 2, figsize=(8, 15))
    f.tight_layout()
    f.set_figheight(30)
    f.set_figwidth(14)
    plt.rcParams.update({'font.size': 18})

    for i, col in enumerate(col_column):
        sns.histplot(data[col], kde=True, bins=24, ax=ax[i, 0])
        sns.boxplot(data[col], ax=ax[i, 1])

        ax[i, 0].set_xlabel(col)
        ax[i, 1].set_xlabel(col)
        ax[i, 0].set_ylabel('Количество')
    plt.suptitle("Гистограмма и ящик с усами для количесвтенных данных",
                 fontsize=22, y=1.01)
    plt.show()
    # f.savefig(filename)


def cat_graph(df, cat_feat, filename='project_cat.png'):
    '''
    Функция отрисовки круговых диаграмм для категориальных переменных.
    На вход: исходная таблица и список категориальных переменных.
    На выходе: графики
    '''

    cols = 2
    rows = int(np.ceil(len(cat_feat) / cols))

    fig, axs = plt.subplots(rows, cols, figsize=(30, 30))
    plt.tight_layout()

    count = -1
    for i in range(rows):
        for x in range(cols):
            count += 1
            col = cat_feat[count]
            df1 = pd.DataFrame(df.groupby([col])[col].count())
            axs[i, x].pie(x=df1[col],
                          labels=df1.index,
                          autopct='%1.1f%%', )
            axs[i, x].title.set_text(str(col))
            if count == len(cat_feat) - 1:
              break
        if count == len(cat_feat) - 1:
              break

    plt.suptitle('Круговые диаграммы категориальных признаков', fontsize=20,
                 y=1.05)

    plt.show()
    # fig.savefig(filename)


def normal_sum_test(x, Ptest):
    """
    Функция проверяет нормальность/ненормальность распределения
    по сумме 3-х тестов: Шапиро, Андерсона-Дарлинга, Харке-Бера

    На выходе:
    1 - ненормальное распределение
    0 - нормальное распредление

    Принцип большинства заложен.
    Внутри есть функция расчёта критерия Андерсона, исходя из уровня значимсоти
    """

    def anderson_chois_sig(A, Ptest):
        if Ptest == 0.05:
            ander = A[2]
        elif Ptest == 0.01:
            ander = A[4]
        return ander

    def normalnost_anderson(x, Ptest):
        A2, crit, sig = anderson(x, dist='norm')
        ad_pass = (A2 < crit)
        norm = anderson_chois_sig(ad_pass, Ptest)
        if norm == False:
            return 1
        return 0

    # print(x)
    p_shapiro = shapiro(x)[1]
    p_jarque = jarque_bera(x)[1]

    if p_shapiro < Ptest:
        p_shapiros = 1
    else:
        p_shapiros = 0

    if p_jarque < Ptest:
        p_jarques = 1
    else:
        p_jarques = 0

    p_anderson = normalnost_anderson(x,
                                     Ptest)  # 1 - ненормальное, 0 - нормальное

    p_sum = p_shapiros + p_anderson + p_jarques

    if (p_sum > 1):
        return 1
    else:
        return 0


def convert_to_pep(df):
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.lower()
    return df


def process_feelling(x):
    if x < 0:
        return "неизвестно"
    if x == 0:
        return "неприемлемо"
    return "приемлемо"


def process_comfort(x):
    if x < 1:
        return "неизвестно"
    if x < 2:
        return "очень неудобно"
    if x < 3:
        return "неудобно"
    if x <= 4:
        return "нормально"
    if x <= 5:
        return "комфортно"
    return "очень комфортно"


def process_closing(x):
    if x < 0:
        return "неизвестно"
    if x == 0:
        return "открыто"
    return "закрыто"


def process_connected(x):
    if x < 0:
        return "неизвестно"
    if x == 0:
        return "выключен"
    return "включен"


def convert_to_celsius(x):
    celsius = (x - 32) * 5 / 9.0
    return celsius


def stage1(df):
    # конвертируем в pep8
    df = convert_to_pep(df)

    # типы столбцов
    df["год"] = df["год"].astype(int).astype("object")
    df["ощущение_температуры_(bool)"] = df["ощущение_температуры_(bool)"].astype(int).astype("object")

    # удаление строк с отсутствующим возрастом
    df = df.dropna(subset=["возраст"])
    df["возраст"] = df["возраст"].astype(int)

    col = "режим_при_смешанном_типе_охлаждения"
    df[col] = df[col].fillna("отсутствует")

    col = "способ_обогрева"
    df[col] = df[col].fillna("отсутствует")

    col = "пол"
    df[col] = df[col].fillna("не указан")

    col = "ощущение_движения_воздуха_(bool)"
    df[col] = df[col].fillna(-1.0)
    df[col] = df[col].apply(process_feelling)
    df[col] = df[col].astype(object)

    col = "оценка_комфорта"
    df[col] = df[col].fillna(df.groupby(["способ_охлаждения"])[col].
                             transform("mean"))
    df["оценка_комфорта_кат"] = df[col].apply(process_comfort)

    col = "температура_воздуха_на_улице"
    average_col = "среднемесячная_температура_на_улице"
    df[col] = df[col].fillna(df.groupby(["город", "время_года"])[average_col].
                             transform("mean"))

    df.drop(["рост", "вес"], axis=1, inplace=True)

    col = "занавески"
    df[col] = df[col].fillna(-1.0)
    df[col] = df[col].apply(process_closing)
    df[col] = df[col].astype(object)

    col = "вентилятор"
    df[col] = df[col].fillna(-1.0)
    df[col] = df[col].apply(process_connected)
    df[col] = df[col].astype(object)

    col = "окно"
    df[col] = df[col].fillna(-1.0)
    df[col] = df[col].apply(process_closing)
    df[col] = df[col].astype(object)

    col = "двери"
    df[col] = df[col].fillna(-1.0)
    df[col] = df[col].apply(process_closing)
    df[col] = df[col].astype(object)

    col = "отопление"
    df.loc[df["способ_охлаждения"] == "Кондиционирование", col] = 1.0
    df.loc[df["режим_при_смешанном_типе_охлаждения"] == "Кондиционирование",
           col] = 1.0
    df.loc[df["способ_охлаждения"] == "Вентиляция", col] = 0.0
    df.loc[df["режим_при_смешанном_типе_охлаждения"] == "Вентиляция",
           col] = 0.0
    df[col] = df[col].apply(lambda x: "выключено" if x == 0.0 else "включено")
    df[col] = df[col].astype(object)

    # удаляем дубликаты
    df = df.drop_duplicates()

    # удаляем выбросы
    new_name = {"Cубтропический океанический": "Cубтропический океанический",
                "Cубтроп океанич": "Cубтропический океанический",
                "Субтропическое высокогорье": "Субтропическое высокогорье",
                "Тропическая влажная саванна": "Тропическая влажная саванна",
                "Жаркий полузасушливый": "Жаркий полузасушливый",
                "Влажный субтропический муссонный":
                    "Влажный субтропический муссонный"}
    df["климат"] = df["климат"].map(new_name)

    new_name = {"Без изменений": "Без изменений",
                "Теплее": "Теплее",
                "Холоднее": "Холоднее",
                "Холодн": "Холоднее",
                "Тепле": "Теплее"}
    col = "предпочтительное_изменение_температуры"
    df[col] = df[col].map(new_name)

    # убираем выбросы с помощью тройного интерквартильного размаха
    col = "скорость_воздуха"
    q1 = df[col].quantile(0.25)
    q2 = df[col].quantile(0.75)
    delta = q2 - q1
    lower_bound = q1 - 3 * delta
    upper_bound = q2 + 3 * delta
    median = df[col].median()
    df.loc[(df[col] < lower_bound), col] = median
    df.loc[(df[col] > upper_bound), col] = median

    col = "температура_воздуха_в_помещении"
    df[col] = df[col].apply(lambda x: convert_to_celsius(x) if x > 50 else x)

    col = "среднемесячная_температура_на_улице"
    mean_temp = df.groupby("город")[col].mean()
    df.loc[df[col] > 100, col] = df["город"].map(mean_temp)
    df[col] = df[col].apply(lambda x: convert_to_celsius(x) if x > 50 else x)

    return df


def count_of_adv(x):
  if x <= 1:
    return "мало"
  elif x == 2:
    return "средне"
  return "много"


def make_age_cat(x):
    if x <= 44:
        return 'молодой_возраст'
    elif x <= 59:
        return 'средний_возраст'
    return 'пожилой_возраст'


def standart_rh(x):
    if x < 40:
        return "Ниже нормы"
    elif 40 <= x <= 60:
        return "Норма"
    return "Выше нормы"


def corr(df, x, y):  # функция для расчета корреляции между двумя факторами
    if is_numeric_dtype(df[x]) and is_numeric_dtype(df[y]):
        if normal_sum_test(df[x], 0.05) == 1 and normal_sum_test(df[y], 0.05) == 1:
            return round(pearsonr(df[x], df[y])[0], 3)
        return round(spearmanr(df[x], df[y])[0], 3)
    elif is_string_dtype(df[x]) and is_string_dtype(df[y]):
        df1 = df[[x, y]]
        tab = pd.crosstab(df1[x], df1[y])
        return round(math.sqrt(chi2_contingency(tab).statistic / (df1.shape[0] * (min(df1.shape) - 1))), 3)
    elif is_string_dtype(df[y]):
        x, y = y, x
    # расчет корреляционного значения Eta
    cat = df[x].unique()
    in_group = 0
    inter_group = 0
    for i in cat:
        df1 = df[df[x] == i]
        m = df1[y].mean()
        in_group += df1[y].apply(lambda x: (x - m) ** 2).sum()
        inter_group += df1[y].count() * (m - df[y].mean()) ** 2

    return round(inter_group / (in_group + inter_group), 3)


def cheddok(coef, x, y):
    # для оценки силы связи будем использовать шкалу Чеддока
    if 0.5 <= abs(coef) <= 0.7:
        return f"Заметная связь между факторами {x} и {y}. Коэффициент корреляции = {coef}"
    elif 0.7 < abs(coef) <= 0.9:
        return f"Сильная связь между факторами {x} и {y}. Коэффициент корреляции = {coef}"
    elif 0.9 < abs(coef) < 1:
        return f"Очень сильная связь между факторами {x} и {y}. Коэффициент корреляции = {coef}"


def research(df):
    col = "количество_рекламаций_кат"
    df[col] = df["количество_рекламаций"].apply(count_of_adv)

    col = 'возраст_кат'
    df[col] = df['возраст'].apply(make_age_cat)

    # средний возраст по полу и стране
    avg_age = pd.DataFrame(df.groupby(["пол", "страна"])["возраст"]
                 .apply(lambda x: x.sum() // x.count())).rename(columns=
                  {"возраст": "средний_возраст"})

    # медианное значение температуры и влажности для каждого типа охлаждения
    med_type_of_cool = pd.DataFrame(df.groupby("способ_охлаждения")
    [["температура_воздуха_в_помещении", "rh"]]
                 .apply(lambda x: x.median())).rename(columns=
                  {"температура_воздуха_в_помещении": "медианная_температура",
                     "rh": "медианная_влажность"})

    # средняя комфортная температура в зависимости от возрастной категории
    avg_comf_temp_age = pd.DataFrame(df[df["предпочтительное_изменение_температуры"]
                                    == "Без изменений"].groupby("возраст_кат")
                                    ["температура_воздуха_в_помещении"]
                                 .apply(lambda x: round(x.sum() / x.count(), 1))).rename(
        columns={"температура_воздуха_в_помещении":
                 "средняя_комфортная_температура"})

    # процент удовлетворенных температурой респондетов по стране и полу
    df["процент_удовлетворенных_температурой_респондетов_по_полу_и_стране"] = df.groupby(
        ["страна", "пол"])["предпочтительное_изменение_температуры"].transform(
            lambda x: round(x.value_counts(normalize=True)[0] * 100, 1))

    '''
    сводная таблица, в которой данные сгруппированы по стране, полу,
    возрастной группе и посчитаны средняя температура воздуха в помещении,
    на улице и средняя относительная влажность для каждой из этих групп.
    '''
    avg_temp_rh = df.groupby(["страна", "пол", "возраст_кат"])[["температура_воздуха_в_помещении", "температура_воздуха_на_улице" , "rh"]].apply(lambda x:
                                                    round(x.sum() / x.count(), 1)).rename(
                                                        columns = {
                                                            "температура_воздуха_в_помещении": "средняя_теспература_воздуха_в_помещении",
                                                            "температура_воздуха_на_улице": "средняя_температура_воздуха_на_улице",
                                                            "rh": "средняя_относительная_влажность"
                                                            }
                                                    )
    # стандартная комфортная относительная влажность в офисах по СНиП: 40-60 %
    # создадим категориальный столбец для влажности
    df["rh_кат"] = df["rh"].apply(standart_rh)

    # исследуем корреляцию между параметрами
    list_of_corr = []  # список всех взаимосвязей, которые имеют значение
    for i in range(len(df.columns) - 1):
        for j in range(i + 1, len(df.columns)):
            coef = corr(df, df.columns[i], df.columns[j])
            if 1 > abs(coef) >= 0.5:
                list_of_corr.append(cheddok(coef, df.columns[i], df.columns[j]))
    return df, avg_age, med_type_of_cool, avg_comf_temp_age, avg_temp_rh, list_of_corr


df = pd.read_csv("data.csv", sep=";", decimal=".")

df = stage1(df)
df, avg_age, med_type_of_cool, avg_comf_temp_age, avg_temp_rh, list_of_corr = research(df)







